# Assignment - 2: Data Representation and Point Cloud Operations

Team Name: \<team name here\> 

Roll number: \<Roll number here (in sorted order)\>

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 26/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.

# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import struct
import open3d
import cv2
import io

# Introduction to types of Transformations and Homogeneous coordinates

In robotics applications, it is inevitable to keep track of the frames of multiple objects/worlds. These frames can be transformations from one coordinate frame to the other. **Homogeneous coordinates** help in keeping track of various coordinate frames and allow performing composition of various transforms. We will first try to understand between types of transformations and their invariant properties.
1. What is the difference between Affine, Similarity, and Euclidean transform? What are the invariant properities of each type of transform?
2. Watch this [video](https://www.youtube.com/watch?v=PvEl63t-opM) to briefly understand homogeneous coordinates. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity? 
3. Using homogeneous coordinates we can represent different types of transformation as point transforms vs. frame transforms. Concatenation of transforms (whether you post multiply transformation matrices or pre-multiply transformation matrices) depends on the problem and how you are viewing it. Try to understand the difference between frame vs. point transformations from this [video](https://youtu.be/Za7Sdegf8m8?t=1834). Let's assume that our camera and world frames are coinciding with each other. We need to estimate the camera to world **frame** transformation matrix after applying the transformations defined below in terms of $T_i$.We apply **frame** transform to move the camera in the world in the following order:
    1. $T_1$ from the camera coordinate frame.
    2. $T_2$ from the world coordinate frame.
    3. $T_3$ from the world coordinate frame.
    4. $T_4$ from the camera coordinate frame.
    5. $T_5$ from the camera coordinate frame.


<b>1.</b><br>
   <b> 1.1.</b> Euclidean transform preserves angle and lengths, similarity transform preserves ratios of lengths, areas, and angles while affine transform preserves parallelism.<br>
   <b> 1.2.</b>In terms of degrees of freedom, Affine transform has 6 DOF, Similarity transform has 4 DOF while euclidean transform has only 3 DOF.<br>
<b>2.</b> Point where parallel lines appear to converge is to be taken as point at infinity(also called vanishing point). Perspective transform can be used to transform a point at infinity to a point at finite distance as follows: 
$$\begin{bmatrix} 0 & 0 & 1 & 0\end{bmatrix} \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & x \end{bmatrix} = \begin{bmatrix} 0 & 0 & 1/x & 1\end{bmatrix}$$

# Visualise the Data

Point clouds are a collection of points that represent a 3D shape or feature. Each point has its own set of X, Y and Z coordinates and in some cases additional attributes. A popular way to obtain this is by photogrammetry, though here we will use LiDAR data.

LiDAR is a remote sensing process which collects measurements used to create 3D models and maps of objects and environments. Using ultraviolet, visible, or near-infrared light, LiDAR gauges spatial relationships and shapes by measuring the time it takes for signals to bounce off objects and return to the scanner.

1. Download the data from [here](https://iiitaphyd-my.sharepoint.com/:f:/g/personal/venkata_surya_students_iiit_ac_in/EnYAMaTVIhJItzKYqtahE30BRKB6p6UfHN3TyJzvo6Mw0g?e=PegWds). It contains the LIDAR sensor output and odometry information per frame.

    The .bin files contain the 3D point cloud captured by the LIDAR in this format - x, y, z, and reflectance. 

    The odometry information is given in the `odometry.txt` file, which is a 12 element vector. Reshape each of the first 77 rows to a 3x4 matrix to obtain the pose.
    

2. Obtain the point cloud from this and visualise for 1-2 frames.

# Answers:

### Part 1

In [27]:
lines = []
with io.open("data/odometry.txt", mode="r", encoding="utf-8") as file:
    for row in file:
        l = row.split()
        lines.append(l)
# print(lines)

In [27]:
#Convert to float
val = np.array(lines).astype('float')
print(val)
print(val.shape)

[[ 1.000000e+00  9.043683e-12  2.326809e-11 ...  2.392370e-10
   9.999999e-01 -2.220446e-16]
 [ 9.990498e-01 -1.649780e-03  4.355194e-02 ...  2.576529e-03
   9.990480e-01  1.000725e+00]
 [ 9.960495e-01 -5.367949e-03  8.863831e-02 ...  7.485875e-03
   9.960394e-01  1.995659e+00]
 ...
 [ 8.521727e-01 -2.191437e-02  5.228017e-01 ...  6.730095e-02
   8.512152e-01 -1.048145e+03]
 [ 8.510645e-01 -2.217815e-02  5.245927e-01 ...  6.729994e-02
   8.501262e-01 -1.046739e+03]
 [ 8.499437e-01 -2.495543e-02  5.262823e-01 ...  6.530726e-02
   8.492505e-01 -1.045291e+03]]
(1101, 12)


In [31]:
#Obtaining the first 77 rows
odometry = val[:77, :]
print(odometry.shape)
# print(odometry)

(77, 12)


In [39]:
# Reshape into 3x4 matrix
Pose = np.zeros((77,3,4))
for i in range(odometry.shape[0]):
    p = np.reshape(odometry[i], (3, 4))
    Pose[i] = p
# print(Pose[2])

### Part 2

In [32]:
# Read LiDAR sensor data and convert to PCD format for easier visualisation
def BinToPCD(data):
    PCD_point=[]
    with open(data,"rb") as file:
        val = file.read(16)
        while val:
            x,y,z,reflectance = struct.unpack("ffff",val)
            PCD_point.append([x,y,z])
            val = file.read(16)
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(np.array(PCD_point))
    return pcd

In [33]:
z = []
for j in range(1,8):
    for i in range(9):
        path = "data/LiDAR/0000"+str(j)+str(i)+".bin"
        z1 = BinToPCD(path)
        z.append(z1)
# print(z)
# z1 = convert_kitti_bin_to_pcd("data/LiDAR/000010.bin")
# z2 = convert_kitti_bin_to_pcd("data/LiDAR/000039.bin")
# z3 = convert_kitti_bin_to_pcd("data/LiDAR/000085.bin")
# z4 = convert_kitti_bin_to_pcd("data/LiDAR/000077.bin")

In [34]:
open3d.visualization.draw_geometries(z)

# Transform 

The point cloud obtained is with respect to the LiDAR frame. The poses however, are in the camera frame. If we want to combine the point clouds from various frames, we need to bring them to the camera frame. 

1. Refer to the image below and apply the required transformation to the point cloud. 

2. Then, register all point clouds into a common reference frame and visualise it (Open3D). It is helpful to use homogeneous coordinates to keep track of the different frames.

3. Write a function to transform the registered point cloud from the world to the $i^{th}$ camera frame, wherein $i$ is the input to the function.

4. \[Bonus\] Move around in the registered point cloud using arrow keys like you would do in a game. For this you will have to regularly transform the entire registered world to your current camera frame and visualize repeatedly. You may choose to avoid visualizing points that are behind the camera in this case as they are not visible from the scene. You may also visualize points at a max depth to make the process easier.

![](./img/transform.png)

# Occupancy Map

Occupancy grid maps are discrete fine grain grid maps. These maps can be either 2-D or 3-D. Each cell in the occupancy grid map contains information on the physical objects present in the corresponding space. Since these maps shed light on what parts of the environment are occupied, and what is not, they are really useful for path planning and navigation.

Occupancy grid maps are probabilistic in nature due to noisy measurements. Each cell can have three states: Occupied, unoccupied, and unknown. For the purpose of this assignment, you can ignore the unknown and work in a binary setting where 1 is occupied and 0 is unoccupied.

1. The task here is to create an occupancy map for each LiDAR scan. You do not need to apply bayesian update rules here, just keep it simple. 

2. Now, using the *registered* point cloud, generate occupancy maps for each frame. What difference do you expect to see between the two methods?

You can mark a cell as occupied based on a threshold of how many different z values are there for a particular (x,y) cell.